# Práctica 2: Limpieza y análisis de datos
## Autores
Hemos realizado esta práctica:
* Ignacio Such Ballester
* Andrés Isidro Fonts Santana

## 1. Descripción del _dataset_
### 1.1 Contexto
Se pretende sacar al mercado un nuevo juego de mesa lo más existoso posible y convertirlo en un bestseller.

Para ello, hemos escogido el _dataset_ [Board Game Data](https://www.kaggle.com/datasets/mrpantherson/board-game-data?select=bgg_db_2018_01.csv), disponible en la plataforma Kaggle.

Este conjunto de datos se ha extraído mediante la API del portal [Board Games Geek](https://boardgamegeek.com/). El _dataset_ se generó en enero de 2018 y contiene datos sobre los primeros 5000 juegos de mesa del _ranking_ de Board Games Geek. 

A través de este set de datos, podemos realizar un análisis profundo del mismo, obteniendo correlaciones, clasificaciones en incluso predicciones para averigurar cómo diseñar nuestro juego de mesa.

Además, se podrá proceder a crear modelos de regresión que permitan predecir si un juego será un bestseller o no en función de sus características y contrastes de hipótesis que ayuden a identificar propiedades interesantes en las muestras.

### 1.2 Descripción de los atributos 
Cada uno de los 5000 registros con que cuenta al _dataset_ viene determinado por 20 attributos:

| Nombre      | Tipo    | Descripción                         | Ejemplo
|:------------|:--------|:------------------------------------|:-------------------------------------------------------------
| rank        | int     | Posición en el _ranking_ de BGG     | 21
| bgg_url     | string  | Link a url de la reseña en BGG      | https://boardgamegeek.com/boardgame/167791/terraforming-mars
| game_id     | string  | Identificador del juego en BGG      | 25613
| names       | string  | Nombre del juego                    | Terraforming Mars
| min_players | int     | Número mínimo de jugadores          | 2
| max_players | int     | Número máximo de jugadores          | 4
| avg_time    | int     | Tiempo medio de partida (minutos)   | 60
| min_time    | int     | Tiempo mínimo de partida (minutos)  | 30
| max_time    | int     | Tiempo máx de partida (minutos)     | 120
| year        | int     | Año de publicación                  | 2014
| avg_rating  | float   | Puntuación media del juego según usuarios de BGG (sobre 10)| 8.0096
| geek_rating | float   | Puntuació de BGG, obtenida a través de un algoritmo de ponderación bayesiana (algoritmo no público) |8.49837
| num_votes   | int     | Número de usuarios que han dado puntuación al juego | 1779
| image_url   | string  | Enlace a la imagen del juego  | https://cf.geekdo-images.com/images/pic361592.jpg
| age         | int     | Edad mínimia recomendada | 12
| mechanic    | string  | Tipo de Mecánicas del juego, separadas por comas | Area Enclosure, Card Drafting, Hand Management, Variable Player Powers, Worker Placement
| owned       | int     | Número de usuarios de BGG que han notificado que poseen el juego | 18217
| category    | string  | Categorías a las que pertenece el juego, separadas por comas | Ancient, Card Game, City Building, Civilization
| designer    | string  | Diseñador/a del juego. Si más de uno, separados por comas | Jamey Stegmaier
| weight      | float   | Grado de complejidad del juego (sobre 5) | 2.394


## 2. Carga de datos y selección
Utilizaremos la librería `pandas` para trabajar con los datos.

In [41]:
import pandas as pd

# Llamamos "bgg" al dataframe creado a partir del dataset
bgg = pd.read_csv('../csv/bgg_db_2018_01.csv',sep=',',encoding='latin-1')

Constatamos que los atributos `bgg_url` y `image_url` no nos van a aportar ningún matiz ni información relevante al estudio que queremos realizar sobre los datos. 

Eliminamos estos datos usando el método `.drop()`. 

In [2]:
# Eliminamos las columnas que no utilizaremos para el análisis
bgg.drop('bgg_url'  , inplace=True, axis=1)
bgg.drop('image_url', inplace=True, axis=1)

## 3. Limpieza de los datos

**NOTAS**

Según este enlace https://boardgamegeek.com/wiki/page/BoardGameGeek_FAQ#toc13 BGG tiene un algoritmo que añade hasta 100 votos virtuales para ponderar los ratings.

En la sección de limpieza deberíamos quedarnos con un subset de registros que cumplan un requisito, por ejemplo, que tengan más de 100, 200 o 300 votos... según veamos

In [ ]:
# Show 5 rows of the dataframe
bgg.head()

# Show number of rows in the dataframe
bgg.shape

In [ ]:
# Show if dataframes has NA values
bgg.isnull().sum()

# Vemos que en el dataset no existen valores nulos

# Show 0 values of the dataframe

In [ ]:
# Show extreme values with 2 decimals
bgg.describe().round(2)

# Aquí se puede ver que para algunas observaciones, encontramos juegos que tienen valores extremos, como son en las variables avg_time, min_time y max_time. También para la variable max_players, se encuentran valores extremos, como son en el caso de que el valor sea 0.

In [ ]:
# Show row where is max value of avg_time
bgg[bgg['avg_time']==bgg['avg_time'].max()]

In [ ]:
# Show rows where max_players equals 0
bgg[bgg['max_players']==0]

In [ ]:
# Show value table of max_players
bgg.max_players.value_counts()

In [ ]:
# Show rows where min_players equals 0
bgg[bgg['min_players']==0]

In [ ]:
# Show value table of min_players
bgg.min_players.value_counts()

Vemos que en diferentes variables tenemos valores extremos y valores que no concuerdan con el dataset, como por ejemplo que el tiempo medio de una partida sean 27000 minutos.
A continuación, modificamos los valores extremos y los sustituimos por los valores medios de la misma variable.

*NOTA*
Sí podemos tener juegos con duración muy extendida, en caso de ser un "Legacy", en que el juego se va desarrollando en modo campaña y se tardan decenas de partidas en acabar el juego. Este tipo de juegos típicamente no tiene rejugabilidad.

Por otro lado, la columna avg_time es igual a max_time, por lo que la retiraría del data set

In [ ]:
# MIN_PLAYERS
# Substitute 0 with min_players mean in min_players column
bgg.min_players.replace(0,1,inplace=True)

#MAX_PLAYERS
# Substitute values higher than 10 with 10 in max_players column
bgg['max_players'].where(bgg['max_players'] < 10, 10, inplace=True)
bgg.max_players.replace(0,1,inplace=True)


#MIN_TIME
bgg['min_time'].where(bgg['min_time'] < 91, 90, inplace=True)
bgg['min_time'].where(bgg['min_time'] > 16, 15, inplace=True)
bgg.min_time.replace(16,15,inplace=True)
bgg.min_time.replace(42,40,inplace=True)

#AVG_TIME

bgg['avg_time'].where(bgg.avg_time.value_counts()==1, bgg.avg_time.mean(), inplace=True)
mask = bgg.avg_time.map(bgg.avg_time.value_counts()) < 5
bgg.avg_time =  bgg.avg_time.mask(mask, bgg.avg_time.mean().round(2))
bgg.avg_time.replace(0,15,inplace=True)

#MAX_TIME
bgg['max_time'].where(bgg['max_time'] < 300, 90, inplace=True)
bgg['max_time'].where(bgg['max_time'] > 10, 10, inplace=True)
bgg['max_time'].where(bgg.max_time.value_counts()==1, bgg.max_time.mean(), inplace=True)
mask = bgg.max_time.map(bgg.max_time.value_counts()) < 5
bgg.max_time =  bgg.max_time.mask(mask, bgg.max_time.mean().round(2))
bgg.max_time.replace(0,15,inplace=True)


#YEAR
bgg['year'].where(bgg['year'] > 1950, 1950, inplace=True)
bgg.year.value_counts()


## 4. Análisis de los datos

*Ideas*
Estudiar la relación entre complejidad (weight) y rating (geek rating y avg rating)

Estudiar la relación entre complejidad (weight) y owned

Estudiar la relación entre rating (geek y avg) y owned

Estudiar qué Mecánicas son las más populares (mechanics vs owned)

Estudiar las categorías más populares (mechanics vs owned)



### 4.x Relación entre Complejidad y Popularidad
Pensamos que la complejidad del juego puede ser un factor clave en cuanto a la popularidad esperada. Analizaremos a continuación la relación entre complejidad (atributo `weight`) y la popularidad (atributo `owned`, esto es, cuántos usuarios han notificado que poseen el juego).

In [63]:
pd.options.plotting.backend = "plotly"
import plotly.express as px

# Filtrar 
ax1 = bgg.head(250).plot.scatter(x='num_votes', y='weight')



In [35]:
ax1

# Zona de test

In [ ]:

plot2 = bgg.head(1000).plot.scatter(x="avg_rating", y="weight")
plot2

In [ ]:
plot3 = bgg.head(250).plot.scatter(x="owned", y="geek_rating")
plot3

In [ ]:
plot4 = bgg.plot.scatter(x="owned", y="num_votes")
plot4

In [74]:

fig = px.scatter(bgg, x='owned', y='num_votes', trendline='ols')
#plot4 = bgg[min_votes].plot.scatter(log_x=True, log_y=True, x="owned", y="num_votes", trendline='ols')
#plot4

ModuleNotFoundError: No module named 'statsmodels'

In [69]:
min_votes = bgg['num_votes'] > 5000

In [70]:
plot5 = bgg[min_votes].plot.scatter(x="weight", y="num_votes", text="game_id")
plot5

In [71]:
df = bgg[min_votes]

fig = px.histogram(df, x="weight", y="num_votes", marginal="box",
                   hover_data=df.columns)
fig.show()

In [49]:
plot6 = bgg[min_votes].plot.scatter(x="avg_rating", y="num_votes", text="game_id")
plot6

In [59]:
plot7 = bgg[min_votes].plot.distplot(x="avg_rating", y="num_votes")

print(bgg[min_votes].avg_rating.mean())
plot7

AttributeError: 'PlotAccessor' object has no attribute 'distplot'

In [72]:
plot6 = bgg[min_votes].plot.scatter(x="num_votes", y="geek_rating")

#Çplot6

df = bgg[min_votes]
fig = px.histogram(df, x="geek_rating", y="num_votes", marginal="violin",
                   hover_data=df.columns)
fig.show()

In [ ]:
import pandas as pd
from IPython.display import display, HTML

# Assuming that dataframes df1 and df2 are already defined:

print( "Dataframe 2:")
display(HTML(bgg.head().to_html()))


#bgg=pd.read_csv('../csv/bgg_db_2018_01.csv',sep=',',encoding='latin-1')


## Referencias consultadas


Board Game Rank \[en línea\] \[fecha de consulta: 30 de mayo de 2022\]. Disponible en: https://boardgamegeek.com/browse/boardgame?sort=rank&sortdir=desc

BoardGameGeek FAQ \[en línea\] \[fecha de consulta: 31 de mayo de 2022\]. Disponible en: https://boardgamegeek.com/wiki/page/BoardGameGeek_FAQ#toc13

BoardGameWiki. Weight \[en línea\] \[fecha de consulta: 31 de mayo de 2022\]. Disponible en: https://boardgamegeek.com/wiki/page/Weight

How to delete a column in pandas \[en línea\] \[fecha de consulta: 01 de junio de 2022\]. Disponible en: https://www.educative.io/edpresso/how-to-delete-a-column-in-pandas

Pandas Plotting Backend in Python \[en línea\] \[fecha de consulta: 02 de junio de 2022\]. Disponible en: https://plotly.com/python/pandas-backend/

Working with Markdown tables in GitHub \[en línea\] \[fecha de consulta: 03 de junio de 2022\]. Disponible en: https://www.pluralsight.com/guides/working-tables-github-markdown

Practical Business Python. Overview of Pandas Data Types \[en línea\] \[fecha de consulta: 3 de junio de 2022\]. Disponible en: https://pbpython.com/pandas_dtypes.html

